# Imports

## Packages

In [1]:
import pandas as pd
import holidays
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
#import xgboost as xg
from entsoe import EntsoePandasClient
from statsmodels.graphics import tsaplots
from sklearn.preprocessing import StandardScaler, PowerTransformer

import itertools

# Prophet model for time series forecast
# !pip install yfinance prophet
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

import warnings
warnings.filterwarnings('ignore')

C:\Users\Elena\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Elena\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\Elena\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Data

In [2]:
df = pd.read_csv('DK_2.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format = '%Y %m %d %H:%M:%S')
# df['Date'] = pd.to_datetime(df['Date'], format = '%Y %m %d')

In [3]:
df.head().T

,0,1,2,3,4
Timestamp,2018-01-01 00:00:00,2018-01-01 01:00:00,2018-01-01 02:00:00,2018-01-01 03:00:00,2018-01-01 04:00:00
Date,2018-01-01,2018-01-01,2018-01-01,2018-01-01,2018-01-01
TTF,NaN,NaN,NaN,NaN,NaN
CO2,NaN,NaN,NaN,NaN,NaN
Day-ahead prices,26.33,26.43,26.1,24.7,24.74
Forecasted Load,3422.0,3289.0,3157.0,3025.0,2939.0
Actual Load,3421.0,3308.0,3118.0,3018.0,2916.0
Solar,0.0,0.0,0.0,0.0,0.0
Wind Offshore,783.0,893.0,755.0,747.0,886.0
Wind Onshore,1493.0,1481.0,1430.0,1458.0,1472.0


In [4]:
# add year, quarter, month, day, date and hour columns
def get_dt_info(df, dt_col, yr = False, qt = False, mo = False, day = False, date = False, w = False, h = False):
    if yr:
        df['Year'] = df[dt_col].dt.year
    if qt:
        df['Quarter'] = df[dt_col].dt.quarter
    if mo:
        df['Month'] = df[dt_col].dt.month
    if day:
        df['Day'] = df[dt_col].dt.day
    if date:
        df['Date'] = df[dt_col].dt.date
        df['Date'] = pd.to_datetime(df['Date'], format = '%Y %m %d')
    if w:
        if df[dt_col].dtype == '<M8[ns]':
            df['Week'] = df[dt_col].dt.isocalendar().week
        else:
            print(dt_col,'is not a date!')
    if h:
        df['Hour'] = df[dt_col].dt.hour
        
    return df

In [5]:
df = get_dt_info(df, 'Timestamp', yr = True, qt = True, mo = True, day = True, date = True, h = True)
df = get_dt_info(df, 'Date', w = True)
df.head().T

,0,1,2,3,4
Timestamp,2018-01-01 00:00:00,2018-01-01 01:00:00,2018-01-01 02:00:00,2018-01-01 03:00:00,2018-01-01 04:00:00
Date,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00,2018-01-01 00:00:00
TTF,NaN,NaN,NaN,NaN,NaN
CO2,NaN,NaN,NaN,NaN,NaN
Day-ahead prices,26.33,26.43,26.1,24.7,24.74
Forecasted Load,3422.0,3289.0,3157.0,3025.0,2939.0
Actual Load,3421.0,3308.0,3118.0,3018.0,2916.0
Solar,0.0,0.0,0.0,0.0,0.0
Wind Offshore,783.0,893.0,755.0,747.0,886.0
Wind Onshore,1493.0,1481.0,1430.0,1458.0,1472.0


In [6]:
# calendar of holidays in DK
dk_bus = pd.tseries.offsets.CustomBusinessDay(calendar = holidays.DK())

# range of business days, excluding weekends and holidays
dk_bus_days = pd.bdate_range(min(df['Date']), max(df['Date']), freq = dk_bus)
df['business'] = df['Date'].isin(dk_bus_days)

In [7]:
# check for NaN values
for col in df.columns:
    a = df[col].isna().sum()
    if a > 0:
        b = df[(df[col].isna()) & (df['business'] == True)]
        print(col, a, 'of which', len(b), 'in', len(b['Date'].unique()),'business days')

TTF 13512 of which 1008 in 42 business days
CO2 12936 of which 432 in 18 business days
Day-ahead prices 23 of which 0 in 0 business days
Forecasted Load 26 of which 2 in 1 business days
Actual Load 26 of which 2 in 1 business days
Solar 456 of which 310 in 22 business days
Wind Offshore 120 of which 72 in 5 business days
Wind Onshore 168 of which 120 in 8 business days
Wind Total 168 of which 120 in 8 business days


In [8]:
# in most cases, 'TTF' and 'CO2' NaN are in non-business days, where the price is the same as the last business day --> ffill
for col in ['TTF', 'CO2']:
    df[col].fillna(method = 'ffill', inplace = True)

In [9]:
# check for NaN values
for col in df.columns:
    a = df[col].isna().sum()
    if a > 0:
        b = df[(df[col].isna()) & (df['business'])]
        print(col, a, 'of which', len(b), 'in', len(b['Date'].unique()),'business days')

TTF 24 of which 24 in 1 business days
CO2 24 of which 24 in 1 business days
Day-ahead prices 23 of which 0 in 0 business days
Forecasted Load 26 of which 2 in 1 business days
Actual Load 26 of which 2 in 1 business days
Solar 456 of which 310 in 22 business days
Wind Offshore 120 of which 72 in 5 business days
Wind Onshore 168 of which 120 in 8 business days
Wind Total 168 of which 120 in 8 business days


In [10]:
idx_drop = []
for col in ['TTF', 'CO2', 'Day-ahead prices']:
    print(col, df[df[col].isna()]['Date'].unique())
    idx_drop.append(df[df[col].isna()]['Date'].unique()[0])

idx_drop = set(idx_drop)

TTF ['2018-01-01T00:00:00.000000000']
CO2 ['2018-01-01T00:00:00.000000000']
Day-ahead prices ['2022-12-31T00:00:00.000000000']


In [11]:
# some missing are in the 1st and last day of the whole dataset, all observations for those days are dropped
df.drop(df[df['Date'].isin(idx_drop)].index, inplace = True)

In [12]:
# check for NaN values
for col in df.columns:
    a = df[col].isna().sum()
    if a > 0:
        b = df[df[col].isna()]['Date'].unique()
        print(col, a, '\nbusiness days', len(b), b)

Forecasted Load 2 
business days 1 ['2020-09-09T00:00:00.000000000']
Actual Load 2 
business days 1 ['2020-09-09T00:00:00.000000000']
Solar 432 
business days 32 ['2020-03-01T00:00:00.000000000' '2020-03-02T00:00:00.000000000'
 '2020-07-15T00:00:00.000000000' '2020-07-16T00:00:00.000000000'
 '2020-08-03T00:00:00.000000000' '2020-08-04T00:00:00.000000000'
 '2021-10-07T00:00:00.000000000' '2021-10-08T00:00:00.000000000'
 '2021-10-13T00:00:00.000000000' '2021-10-14T00:00:00.000000000'
 '2022-04-30T00:00:00.000000000' '2022-05-01T00:00:00.000000000'
 '2022-07-29T00:00:00.000000000' '2022-07-30T00:00:00.000000000'
 '2022-08-11T00:00:00.000000000' '2022-08-12T00:00:00.000000000'
 '2022-08-13T00:00:00.000000000' '2022-08-14T00:00:00.000000000'
 '2022-08-16T00:00:00.000000000' '2022-08-17T00:00:00.000000000'
 '2022-08-24T00:00:00.000000000' '2022-08-25T00:00:00.000000000'
 '2022-08-28T00:00:00.000000000' '2022-08-29T00:00:00.000000000'
 '2022-08-31T00:00:00.000000000' '2022-09-01T00:00:00.0000

In [13]:
df_temp = df.copy()
df_temp[(df_temp['Hour'] == 17) & ('2020-09-03'< df_temp['Date']) & (df_temp['Date'] <'2020-09-15')]

,Timestamp,Date,TTF,CO2,Day-ahead prices,Forecasted Load,Actual Load,Solar,Wind Offshore,Wind Onshore,Wind Total,Year,Quarter,Month,Day,Hour,Week,business
23465,2020-09-04 17:00:00,2020-09-04,11.680,28.13,47.95,4108.0,4151.0,90.0,924.0,1520.0,2444.0,2020,3,9,4,17,36,True
23489,2020-09-05 17:00:00,2020-09-05,11.680,28.13,37.29,3789.0,3807.0,88.0,1111.0,1877.0,2988.0,2020,3,9,5,17,36,False
23513,2020-09-06 17:00:00,2020-09-06,11.680,28.13,42.01,3837.0,3881.0,105.0,785.0,1499.0,2284.0,2020,3,9,6,17,36,False
23537,2020-09-07 17:00:00,2020-09-07,11.680,27.81,56.12,4312.0,4348.0,99.0,917.0,1518.0,2435.0,2020,3,9,7,17,37,True
23561,2020-09-08 17:00:00,2020-09-08,10.800,27.53,49.68,4300.0,4300.0,100.0,967.0,1506.0,2473.0,2020,3,9,8,17,37,True
23585,2020-09-09 17:00:00,2020-09-09,10.735,27.94,46.65,NaN,NaN,77.0,1204.0,2360.0,3564.0,2020,3,9,9,17,37,True
23609,2020-09-10 17:00:00,2020-09-10,10.515,29.15,65.81,4409.0,4263.0,95.0,398.0,865.0,1263.0,2020,3,9,10,17,37,True
23633,2020-09-11 17:00:00,2020-09-11,10.495,28.99,56.76,4116.0,4185.0,76.0,869.0,1508.0,2377.0,2020,3,9,11,17,37,True
23657,2020-09-12 17:00:00,2020-09-12,10.495,28.99,33.02,3949.0,3948.0,73.0,1241.0,2455.0,3696.0,2020,3,9,12,17,37,False
23681,2020-09-13 17:00:00,2020-09-13,10.495,28.99,30.34,4018.0,4125.0,61.0,1205.0,2274.0,3479.0,2020,3,9,13,17,37,False


In [14]:
for i in range(24):
    check = df_temp[df_temp['Hour'] == i].rolling(4, min_periods = 1, closed = 'left', axis = 'rows').mean()
#     df_temp[df_temp['Hour'] == i].fillna(value = -1, inplace = True)
    df_temp.fillna(value = check, inplace = True)

In [15]:
df_temp.iloc[23555:23565, :]

,Timestamp,Date,TTF,CO2,Day-ahead prices,Forecasted Load,Actual Load,Solar,Wind Offshore,Wind Onshore,Wind Total,Year,Quarter,Month,Day,Hour,Week,business
23579,2020-09-09 11:00:00,2020-09-09,10.735,27.94,29.82,4639.0,4625.0,146.0,1094.0,2150.0,3244.0,2020,3,9,9,11,37,True
23580,2020-09-09 12:00:00,2020-09-09,10.735,27.94,29.79,4641.0,4624.0,170.0,1020.0,2099.0,3119.0,2020,3,9,9,12,37,True
23581,2020-09-09 13:00:00,2020-09-09,10.735,27.94,29.21,4554.0,4549.0,171.0,973.0,2183.0,3156.0,2020,3,9,9,13,37,True
23582,2020-09-09 14:00:00,2020-09-09,10.735,27.94,29.17,4401.0,4390.0,196.0,1080.0,2064.0,3144.0,2020,3,9,9,14,37,True
23583,2020-09-09 15:00:00,2020-09-09,10.735,27.94,30.41,4386.0,4374.0,207.0,1178.0,2259.0,3437.0,2020,3,9,9,15,37,True
23584,2020-09-09 16:00:00,2020-09-09,10.735,27.94,32.45,4094.5,4123.0,157.0,1206.0,2286.0,3492.0,2020,3,9,9,16,37,True
23585,2020-09-09 17:00:00,2020-09-09,10.735,27.94,46.65,4059.5,4084.0,77.0,1204.0,2360.0,3564.0,2020,3,9,9,17,37,True
23586,2020-09-09 18:00:00,2020-09-09,10.735,27.94,46.88,4188.0,4184.0,12.0,1201.0,2181.0,3382.0,2020,3,9,9,18,37,True
23587,2020-09-09 19:00:00,2020-09-09,10.735,27.94,32.68,4060.0,4107.0,0.0,1227.0,2066.0,3293.0,2020,3,9,9,19,37,True
23588,2020-09-09 20:00:00,2020-09-09,10.735,27.94,28.59,3809.0,3890.0,0.0,1307.0,2179.0,3486.0,2020,3,9,9,20,37,True


In [16]:
# check for NaN values
for col in df_temp.columns:
    a = df_temp[col].isna().sum()
    if a > 0:
        b = df_temp[df_temp[col].isna()]['Date'].unique()
        print(col, a, '\nbusiness days', len(b), b)

In [17]:
df_temp.to_csv('DK_2_filled.csv', index = False)

# Normalization and Standardization of Columns
We are going to use biased normalization method of sk-learn library. The reason for that is that it reduces the error 
of outliers, especially since the denominator of standard deviation equals to N instead of N-1 or N-m.

We saw on sk learn that the biased way of standardization of Sk Learn, StandardScaller does not affect the outputs of the models
and other statistics that derive from the processed Dataframe.

In [ ]:
df[20:30]

## StandardScaler()

In [ ]:
scaler = StandardScaler()

df_stand = df.copy()

# dropping the target 'Day-ahead prices'
df_stand = df_stand.drop(['Day-ahead prices'], axis=1)

df_stand[20:30]

In [ ]:
columns_no_std = ['Timestamp', 'Date', 'Year','Month', 'Day', 'Hour']
columns_std = [col for col in df_stand.columns if col not in columns_no_std]
columns_std

In [ ]:
df_stand = pd.DataFrame(data = scaler.fit_transform(df_stand[columns_std]), columns = columns_std)

In [ ]:
df_stand.head()

In [ ]:
df_stand.isnull().sum()

## Power Scaler yeo-johnson methond

In [ ]:
scaler2 = PowerTransformer(method = 'yeo-johnson')

df_Power = df.copy()
df_Power[20:30]

In [ ]:
(df_Power[columns_std])[20:30]

In [ ]:
scaler2.fit_transform(df_Power[columns_std])

In [ ]:
df_Power = pd.DataFrame(scaler2.fit_transform(df_Power[columns_std]))
df_Power

#df_Power[col_names] = scaler.fit_transform(features.values)
#print(type(dfPower[5]))

df_Power[5].plot.hist()

## Log transform

In [ ]:
df_log = df[df['Day-ahead prices'] > 0].copy()
df_log['Day-ahead prices log'] = np.log(df_log['Day-ahead prices'])

In [ ]:
df_log['Day-ahead prices log'].plot.hist(bins = 100)
plt.xlabel('Price\n€/MWh')
plt.title('Day-ahead prices')
plt.show()

## Prophet

In [ ]:
#idea if possible corelating electricity prices to company stock value, ex. BP petrol and electricity companies

In [ ]:
# WITH LOG OF PRICES!!
df_prophet = df_log.copy()
cols_drop = [col for col in df_prophet if col not in ['Day-ahead prices log', 'Timestamp']]
df_prophet.drop(cols_drop, axis = 1, inplace = True)
cols_map = {'Day-ahead prices log':'y',
            'Timestamp':'ds'}
df_prophet.rename(columns = cols_map, inplace = True)

df_prophet.head()

In [ ]:
# Initiate the model
baseline_model = Prophet()

# Fit the model on the training dataset
baseline_model.fit(df_prophet)

# # Cross validation
# baseline_model_cv = cross_validation(model=baseline_model, initial='365 days', period='30 days', horizon = '30 days', parallel="processes")
# baseline_model_cv.head()

# # Model performance metrics
# baseline_model_p = performance_metrics(baseline_model_cv, rolling_window=1)
# baseline_model_p.head()

# # Get the performance value
# baseline_model_p['mape'].values[0]

In [ ]:
df_prophet['ds'].tail()

In [ ]:
start = pd.to_datetime('2022-09-30 00:00:00')
end = pd.to_datetime('2023-01-01 23:59:00')
fut = pd.DataFrame()
fut['ds'] = pd.date_range(start, end, freq = 'H')

In [ ]:
fut.info()

In [ ]:
forecast = baseline_model.predict(fut)
forecast.tail()

In [ ]:
start = pd.to_datetime('2022-09-30 00:00:00')
end = pd.to_datetime('2022-12-31 00:00:00')
timespan = pd.DataFrame()
timespan['ds'] = pd.date_range(start, end, freq = 'H')

In [ ]:
df_prophet

In [ ]:
timespan['y'] = np.array(df_prophet[(df_prophet['ds'] >= start) & (df_prophet['ds'] <= end)]['y'])

In [ ]:
timespan

In [ ]:
plt.plot(fut, forecast['yhat'], label = 'Future data-Predicted', linewidth = 0.5)
plt.scatter(timespan['ds'], timespan['y'], label = 'Current data-Real', s = 0.5, c= 'k')
# plt.scatter(x = df_prophet['ds'], y = current_forecast['yhat'], label = 'Current data-Predicted')
plt.legend()
plt.show()

## Spliting the Data to Train and Test sets

It is not recomended to do a random split of Time series train and test data. But there is a way to split it by folds that use periodical splits while keeping the time series properties that help us predict the trends and data.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

n_splits = 5

tscv = TimeSeriesSplit(n_splits)